# Деревья решений и случайный лес

## Введение

Мы рассмотрели один из наиболее простых интуитивно понятных методов машинного обучения --- метод $k$ ближайших соседей. Несмотря на очевидные примущества (простая реализация, интерпретируемость, малое число параметров, неплохо изучен теоретически), он имеет и недостатки: 

- если число соседей велико (100 и больше), а такое часто случается в реальной практике, то алгоритм проигрывает в скорости многим другим;
- метод не способен выделять "хорошие" признаки (можно назначить веса признаков a priori, но алгоритм не делает отбора признаков автоматически, в отличие от некоторых других алгоритмов ML);
- необходимость подбора правильной метрики (выбирать всегда евклидово расстояние --- не лучшая стратегия);
- как правило, плохо работает на данных большой размерности (с большим количеством признаков).

В данной лекции мы познакомимся с ещё одной моделью, обладающей хорошей интерпретируемостью и небольшим числом параметров, при этом устраняющей некоторые из недостатков $kNN$. Речь идёт о деревьях решений и их композициях --- случайных лесах.

## Краткое содержание

- общая идея дерева решений;
- процесс построения дерева, критерии выбора признаков и пороговых значений;
- проблема переобучения и борьба с ним ("стрижка" дерева);
- идея бутстрэпа и бэггинга (bootstrap aggregating);
- о роли случайности и почему случайный лес лучше, чем одно дерево.

## Идея дерева решений

Дерево решений --- модель, в некотором смысле близкая к модели принятия решений человеком. Например, при переходе проезжей части наш мозг мысленно "отвечает на вопросы" вида: какой сигнал светофора горит? есть ли поблизости движущиеся транспортные средства? каково расстояние до них? и так далее. Эти вопросы могут образовывать иерархию. Например, если горит красный свет, мы принимаем решение "не переходить проезжую часть", а в случае зелёного сигнала "задаём дополнительные вопросы" о том, безопасно ли переходить улицу. 

Похожую идею можно использовать (и она действительно использовалась раньше!) для классификации заёмщиков банком. Эта задача называется задачей кредитного скоринга. Сотрудник банка задаёт вопросы заёмщику и по его ответам на определённом этапе решает, целесообразно ли выдать кредит такому заёмщику. Тот же процесс происходит, например, при постановке диагноза у врача. Подобный процесс принятия решения принято визуализировать в виде древовидной структуры (блок-схемы):

![](https://basegroup.ru/sites/default/files/treegraph.gif "Дерево принятия решений")

Модель дерева решений в машинном обучении фактически реализует ту же идею. Поэтапно строится серия логических правил вида "если значение **признака 1** больше 5000, о отнести объект к классу 1", "если значение **признака 2** равно нулю, то проверить значение **признака 3**...". На этом пути естественно возникают следующие вопросы:

- как выбрать признак для ветвления;
- как выбрать пороговое значение для этого признака.

Ответы будут даны ниже.

## Процесс построения дерева решений

**Для построения хорошего (и не очень объёмного) дерева решений важно задать "правильные" вопросы о признаках датасета.** Практичной аналогией здесь представляется игра "Угадай знаменитость". Ведущий загадывает некоторую знаменитость, а участники, задавая вопросы с ответами "да/нет", пытаются отгадать. В данной ситуации задать вопрос "Это Криштиану Роналду?" менее полезно, чем спросить "Это мужчина?". В случае отрицательного ответа на первый вопрос вы сократите множество возможных вариантов всего на единицу. При этом любой ответ на второй вопрос сокращает пространство поиска примерно вдвое. Принятие решений компьютером в таких ситуациях основано на **энтропии**.

### Энтропия

Энтропия --- фундаментальное понятие в теории информации, введённое основоположником этой теории, Клодом Шенноном. Интуитивно под энтропией можно понимать "степень неопределённости" или "степень хаоса" в системе. Именно, чем более упорядоченной и определённой является система, тем меньше её энтропия. Задача дерева решений --- внести порядок в набор значений целевой переменной. Следовательно, их цель --- максимальное **уменьшение энтропии**.

Математически энтропия системы с $N$ возможными состояниями вычисляется по формуле Шеннона
$$
S = -\sum_{i=1}^{N} p_i \log_2 p_i,
$$
где $p_i$ --- вероятность $i$-го состояния системы.

В некотором смысле понятие энтропии обратно понятию информации: **чем меньше энтропия (неопределённость), тем больше у нас информации о системе**. Введём понятие "прироста информации" (information gain) как уменьшения энтропии:
$$
IG = S_0 - S_1,
$$
где $S_0$ --- энтропия начальной системы (до задания вопроса), $S_1$ --- суммарная энтропия образовавшихся систем (после задания вопроса). Хорошая наглядная иллюстрация этого понятия приведена в [статье](https://habr.com/ru/company/ods/blog/322534/#kak-stroitsya-derevo-resheniy).

### Другие эвристики построения дерева

Итак, одним из критериев оценки способа ветвления (выбора признака для ветвления и выбора порога для этого признака) на этапе построения дерева классификации является **энтропийный критерий**, основаный на максимизации прироста информации. Однако существуют и другие подходы:

- **Неопределённость Джини (Gini impurity).** Этот критерий направлен на максимизацию количества пар объектов одного класса, оказавшихся в одном поддереве.
- **Ошибка классификации (misclassification error).** Суть состоит в минимизации числа пар объектов одного класса, попавших при разбиении в разные поддеревья.

На практике последний критерий почти не используется, а неопределённость Джини и энтропйиный критерий работают почти одинаково.

**В задачах регрессии** при построении дерева естественным выглядит критерий ветевления, основанный на **дисперсии целевого признака**: при разбиении мы стремимся получить поддеревья с наименьшей дисперсией целевого признака объектов, находящихся в этом поддереве.

### Обобщённый (жадный) алгоритм построения дерева

Алгоритм построения дерева решений реализует **жадную стратегию**: в каждый момент времени ищется оптимальное на данный момент разбиение на два поддерева, не учитывая возможные последствия такого разбиения. В общем виде алгоритм можно записать следующим образом.

1. Если все объекты из выборки $U$ принадлежат одному классу, то вернуть новую листовую вершину с меткой этого класса и закончить алгоритм. В противном случае перейти к шагу 2.
2. Найти оптимальное разбиение по одному из критерев на два поддерева: $U=U_1\cup U_2$.
3. Если одно из поддеревьев пусто, то вернуть новую листовую вершину с меткой преобладающего класса в $U$. Иначе перейти к шагу 4.
4. Создать новую внутреннюю вершину. Рекурсивно применить алгоритм для левого и правого поддеревьев.

Существует несколько популярных алгоритмов построения деревьев решений: CART, ID3, C4.5 и другие.

### Работа с количественными признаками и выбор порога

О том, как дерево решений выбирает порог для разбиения по значениям количественного призака, описано в [статье](https://habr.com/ru/company/ods/blog/322534/#kak-derevo-resheniy-rabotaet-s-kolichestvennymi-priznakami).

## Проблема переобучения деревьев решений

Переобучение (overfitting) --- проблема всех моделей машинного обучения. Конечно же, деревья решений не исключение. Разберёмся на примере, почему деревья склонны переобучаться.

Представим себе, что мы решаем задачу определения размера футболки (S, M, L, XL, XXL, XXXL) по росту и весу. Два близких по росту и/или весу человека вполне могут носить футболки разного размера, так как мы не учитываем здесь ряд других параметров (пол, ширина талии, плеч и т. д.). Если строить дерево решений "в лоб", т. е. до тех пор, пока в листе не останутся объекты только одного класса, то мы можем получить специфические правила вроде "если рост меньше 171.5 см, но больше 166.5 см, при этом вес от 60 до 70 кг, то размер S". В данном случае два близких по параметрам объекта разных классов порождают всё новые и новые точки ветвления, а в полученных листах, вполне вероятно, окажется лишь по одному объекту. Разумеется, для новых данных такое правило будет работать не всегда. В общем случае мы предпочтём какое-то общее правило любому специфическому, которое "подстроилось" под специфические объекты обучающей выборки.

Каково же решение проблемы? Их несколько.

- **Ограничить максимальную глубину дерева.** Например, мы можем запретить модели строить деревья глубины больше 5 (вполне разумное ограничение для большинства задач).
- **Ограничить минимальное число объектов в листе.** Например, будем объявлять листом любую вершину, которая содержит 5 и менее объектов (с тем чтобы специфические объекты не приводили к появлению новых ветвей дерева). При наличии в листе объектов разных классов выбираем в качестве метки мажоритарный класс (путём простого голосования), в случае задачи регрессии --- медианное или среднее значение целевого признака в листе.
- **Строить специально переобученные деревья, а затем агрегировать (усреднять) их ответы.** Эта идея лежит в основе алгоритма Random Forest (случайный лес), речь о котором пойдёт чуть ниже.

## Бутстрэп, бэггинг и случайный лес

*To pull oneself over a fence by one's bootstraps* --- "перепрыгнуть через забор, потянув себя за ремешки ботинок". Проще говоря, получить что-то полезное "из ничего", или метод барона Мюнхгаузена. Так проще всего охарактеризовать идею статистического метода **"бутстрэп"**, предложенного в 1977 [Брэдли Эфроном](https://ru.wikipedia.org/wiki/%D0%AD%D1%84%D1%80%D0%BE%D0%BD,_%D0%91%D1%80%D1%8D%D0%B4%D0%BB%D0%B8). Опишем его более формально.

Имеем выборку $X$ размера $N$. Будем равномерно и равновероятно выбирать $N$ объектов из этой выборки **с возвращением**. Имеется в виду, что мы выбираем $N$ раз произвольный объект выборки (с вероятностью $1/N$), при этом каждый раз выбираем из одного и того же множества объектов. Понятно, что из-за возвращения среди полученных $N$ объектов, вообще говоря, окажутся повторы. Обозначим новую выборку $X_1$. Повторяя описанную процедуру $M$ раз, можем получить выборки $X_1$, $X_2$, ..., $X_M$, при этом их может быть сколь угодно много, в зависимости от наших целей. Обратим внимание, что исходная выборка могла быть не очень большого размера, чтобы оценивать по ней какие-либо характеристики генеральной совокупности. Метод бутстрэпа позволяет получить достаточное количество выборок, не привлекая к этому ни одного нового объекта!

Метод **бэггинга** (bagging = **b**ootstrap **agg**regat**ing**) был предложен [Лео Брейманом](https://ru.wikipedia.org/wiki/%D0%91%D1%80%D0%B5%D0%B9%D0%BC%D0%B0%D0%BD,_%D0%9B%D0%B5%D0%BE) в 1994. Это один из первых **ансамблевых** методов в машинном обучении. Идея заключается в следующем. Из обучающей выборки $X$ методом бутстрэпа генерируются $M$ выборок $X_1$, ..., $X_M$. Обучим на каждой выборке свой классификатор $a_i(x)$. Итоговый классификатор построим как "усредняющий" ответы всех классификаторов (путём, например, голосования): $a(x)=\frac{1}{M}\sum_{i=1}^{M}a_i(x)$.

![](https://habrastorage.org/getpro/habr/post_images/d2d/b63/6ff/d2db636ff7c5a59f18858bb93f4323db.png)

Можно строго показать, что такое "усреднение" ответов позволяет уменьшить средний квадрат ошибки (в случае задачи регрессии) в $M$ раз. См. вывод в [статье](https://habr.com/ru/company/ods/blog/324402/#begging). То же верно и для классификации.

### Алгоритм Random Forest (случайный лес)

Метод Random Forest --- это **бэггинг над решающими деревьями**. Кроме того, данный метод использует **случайные подпространства**. Опишем основную идею.

Имеем обучающую выборку $X$ размером $l\times D$ ($l$ объектов с $D$ признаками).

- Выбираем количество деревьев в лесу: $N$. 
- Каждое из $N$ деревьев строим на выборке $X_n$ ($n=\overline{1, N})$, полученной с помощью бутстрэпа, при этом дерево строится не на всём признаковом пространстве, а только с использованием $d<D$ признаков. Эти $d$ признаков выбираются случайным образом. Опытным путём получены такие рекомендации: в задачах классификации выбирать $d=\sqrt{D}$, в задачах регрессии $d=D/3$.
- Итоговый классификатор получается путём агрегирования ответов $N$ деревьев (выбором мажоритарного класса для классификации; медианы или среднего для регрессии).

### Основные гиперпараметры случайного леса

Случайные леса для задач классификации и регрессии в библиотеке Scikit-learn представлены классами [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) и [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). Основными гиперпараметрами моделей являются:

- n_estimators (число деревьев в ансамбле);
- criterion (критерий для разбиения выборки в вершине);
- max_features (число признаков для построения одного дерева; см. рекомендацию выше);
- min_samples_leaf (минимальное число объектов в листе; рекомендуется устанавливать равным 1 для классификации и равным 5 для регрессии);
- max_depth (максимальная глубина дерева).

Заметим, что для построения случайного леса деревья **намеренно** делают переобученными. Во-первых, это ускоряет процесс их построения, а во-вторых, уменьшает коррелированность деревьев между собой. От общего переобучения случайный лес "страхует" случайный выбор объектов для обучения (бутстрэп!), случайный выбор подпространств признаков и усреднение ответов большого числа деревьев. С более строгими рассуждениями предлагаем ознакомиться в [статье](https://habr.com/ru/company/ods/blog/324402/#2-sluchaynyy-les).

## Достоинства и недостатки решающих деревьев и случайного леса

Подведём некоторые итоги.

**Pros:**
- RandomForest имеет высокую точность предсказания. Как правило, работает лучше линейных алгоритмов.
- Практически не чувствителен к выбросам в данных из-за случайного сэмлирования (бутстрэп!).
- Не чувствителен к масштабированию признаков, связано с выбором случайных подпространств.
- Одинаково хорошо обрабатывет как непрерывные, так и дискретные (категориальные) признаки.
- Часто не требует тщательной настройки гиперпараметров, хорошо работает "из коробки".
- Способен эффективно обрабатывать данные с большим числом признаков и классов.
- Редко переобучается, на практике добавление деревьев почти всегда только улучшает композицию, но на валидации, после достижения определенного количества деревьев, кривая обучения выходит на асимптоту.
- Для случайного леса существуют методы оценивания значимости отдельных признаков в модели.
- Хорошо работает с пропущенными данными, являясь чуть ли не единственным таким алгоритмом.
- Возможно применение для неразмеченных данных.
- Высокая параллелизуемость и масштабируемость.

**Cons:**
- В отличие от одного дерева, результаты случайного леса сложнее интерпретировать.
- Работает хуже многих линейных методов, когда в выборке очень много разреженных признаков (например, для текстовых данных --- рассмотрим далее).
- Случайный лес не умеет экстраполировать, в отличие от многих других моделей (но это можно считать и плюсом, так как не будет экстремальных значений в случае попадания выброса).
- Для данных, включающих категориальные переменные с различным количеством уровней, случайные леса предвзяты в пользу признаков с большим количеством уровней: когда у признака много уровней, дерево будет сильнее подстраиваться именно под эти признаки, так как на них можно получить более высокое значение оптимизируемого функционала (типа прироста информации).
- Большой размер получающихся моделей. Требуется $O(N\cdot l)$ памяти для хранения модели, где $N$ --- число деревьев, $l$ --- число обучающих объектов.